exp1: max seq, projectors, and single-call classification
literature survey - classification; a little more details

In [0]:
# based on https://www.datacamp.com/tutorial/fine-tuning-llama-3-2

%pip install -U transformers 
%pip install -U datasets 
%pip install -U accelerate 
%pip install -U peft 
%pip install -U trl 
%pip install -U bitsandbytes 
dbutils.library.restartPython()


  Obtaining dependency information for transformers from https://files.pythonhosted.org/packages/f2/3a/8bdab26e09c5a242182b7ba9152e216d5ab4ae2d78c4298eb4872549cd35/transformers-4.47.1-py3-none-any.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.0/44.1 kB ? eta -:--:--
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.1/44.1 kB 1.8 MB/s eta 0:00:00
  Obtaining dependency information for huggingface-hub<1.0,>=0.24.0 from https://files.pythonhosted.org/packages/61/8c/fbdc0a88a622d9fa54e132d7bf3ee03ec602758658a2db5b339a65be2cfe/huggingface_hub-0.27.0-py3-none-any.whl.metadata
  Obtaining dependency information for tokenizers<0.22,>=0.21 from https://files.pythonhosted.org/packages/22/06/69d7ce374747edaf1695a4f61b83570d91cc8bbfc51ccfecf76f56ab4aac/tokenizers-0.21.0-cp39-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.0/10.1 MB ? eta -:--:--
   ━━━━━━━━━━━━━╸━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/10.1 MB 106.0 MB/s eta 0:00:01
  

In [0]:
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    logging,
    LlamaForCausalLM
)
from peft import (
    LoraConfig,
    PeftModel,
    prepare_model_for_kbit_training,
    get_peft_model,
)
import os, torch
from datasets import load_dataset, Dataset
from trl import SFTTrainer, setup_chat_format, SFTConfig
import pandas as pd
from huggingface_hub import login
from sklearn.metrics import precision_score, recall_score, f1_score, roc_curve, auc
import matplotlib.pyplot as plt 
import mlflow
import pandas as pd
import torch
import gc

2024-12-24 03:39:55.594238: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


[2024-12-24 03:40:03,712] [INFO] [real_accelerator.py:203:get_accelerator] Setting ds_accelerator to cuda (auto detect)


df: /root/.triton/autotune: No such file or directory
/usr/bin/ld: cannot find -laio: No such file or directory
collect2: error: ld returned 1 exit status


 [WARNING]  async_io requires the dev libaio .so object and headers but these were not found.
 [WARNING]  async_io: please install the libaio-dev package with apt
 [WARNING]  If libaio is already installed (perhaps from source), try setting the CFLAGS and LDFLAGS environment variables to where it can be found.
 [WARNING]  Please specify the CUTLASS repo directory as environment variable $CUTLASS_PATH
 [WARNING]  NVIDIA Inference is only supported on Ampere and newer architectures
 [WARNING]  sparse_attn requires a torch version >= 1.5 and < 2.0 but detected 2.3
 [WARNING]  using untested triton version (2.3.1), only 1.0.0 is known to be compatible


In [0]:
base_model = "meta-llama/Llama-3.2-1B-Instruct"
base_dir = "123"

In [0]:
torch_dtype = torch.float16
attn_implementation = "eager"

In [0]:
# QLoRA config
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch_dtype,
    bnb_4bit_use_double_quant=True,
)

# Set torch dtype and attention implementation
if torch.cuda.get_device_capability()[0] >= 8:
    %pip install -qqq flash-attn
    torch_dtype = torch.bfloat16
    attn_implementation = "flash_attention_2"
else:
    torch_dtype = torch.float16
    attn_implementation = "eager"

# model = LlamaForCausalLM.from_pretrained(
#     base_model, quantization_config=bnb_config, device_map='auto',attn_implementation=attn_implementation
# )

model = AutoModelForCausalLM.from_pretrained(
    base_model,
    quantization_config=bnb_config,
    device_map="auto",
    attn_implementation=attn_implementation
)

config.json:   0%|          | 0.00/877 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.47G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/189 [00:00<?, ?B/s]

In [0]:
# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(base_model)

if hasattr(tokenizer, "chat_template") and tokenizer.chat_template is not None:
    tokenizer.chat_template = None  # Reset the chat template

model, tokenizer = setup_chat_format(model, tokenizer)

#tokenizer.pad_token = tokenizer.eos_token

tokenizer_config.json:   0%|          | 0.00/54.5k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

The new embeddings will be initialized from a multivariate normal distribution that has old embeddings' mean and covariance. As described in this article: https://nlp.stanford.edu/~johnhew/vocab-expansion.html. To disable this, use `mean_resizing=False`


In [0]:
# adding adapter
# LoRA config
peft_config = LoraConfig(
    r=16,
    lora_alpha=32,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
    #target_modules=['q_proj', 'v_proj'] 
    target_modules = ['q_proj','k_proj','v_proj','o_proj','gate_proj','down_proj','up_proj','lm_head']
)
model = get_peft_model(model, peft_config)

/local_disk0/.ephemeral_nfs/envs/pythonEnv-151f4588-5515-4975-9984-52aca1cc4862/lib/python3.11/site-packages/peft/tuners/tuners_utils.py:543: UserWarning: Model with `tie_word_embeddings=True` and the tied_target_modules=['lm_head'] are part of the adapter. This can lead to complications, for example when merging the adapter or converting your model to formats other than safetensors. See for example https://github.com/huggingface/peft/issues/2018.
  warnings.warn(


In [0]:
# load training data
df = spark.read.parquet("/bdfs/FileStore/churn/test1")
df = df.select("*").toPandas()
df = df.sample(frac=1, random_state=42)

/databricks/spark/python/pyspark/sql/pandas/utils.py:117: UserWarning: The conversion of DecimalType columns is inefficient and may take a long time. Column names: [LOSKEY] If those columns are not necessary, you may consider dropping them or converting to primitive types before the conversion.
  warnings.warn(


In [0]:
# put training data into prompts and set up word limit

def df_parser(raw_pandas, word_limit):
  df = pd.DataFrame() 
  for index, rows in raw_pandas.iterrows():
    chat_history = ""
    for i in range(1,11): #don't include the 1st chat
      if rows[f"transcript_{i}"] != "NA":
        chat_history += f"### New Chat Session ###" + rows[f"transcript_{i}"] + "#########" + "\n\n"
    word_list = chat_history.split()
    word_count = len(word_list)

    if rows["disconnect_date"] != "NA":
      label = "Yes"
    else: label = "No"

    df.at[index, "label"] = label

    if word_count <= word_limit:
      instruction = prompt_template_instruction(chat_history)
      df.at[index, "text"] = instruction + label + "<|eot_id|>"
      df.at[index, "testing_script"] = instruction
    else:
      chat_history = " ".join(word_list[:word_limit])
      instruction = prompt_template_instruction(chat_history)
      df.at[index, "text"] = instruction + label + "<|eot_id|>"
      df.at[index, "testing_script"] = instruction
  return df

def prompt_template_instruction(conversation_history): #llama2 prompt standard: https://www.llama.com/docs/model-cards-and-prompt-formats/meta-llama-2/
  return """<|begin_of_text|><|start_header_id|>system<|end_header_id|>Cutting Knowledge Date: December 2023
Today Date: 23 July 2024

You are a call center analyst. Your task is to identy whether a caller's chat history with agents would indicate their potential disengagement/churn from the service in the future.<|eot_id|><|start_header_id|>user<|end_header_id|>"""  

In [0]:
df_total = df_parser(df,word_limit=12000)
df = df_total[0:5000]
df_inference = df_total[5000:]
dataset = Dataset.from_pandas(df).train_test_split(test_size=0.2, seed=42)

In [0]:
gc.collect()

torch.cuda.empty_cache()

In [0]:
per_device_train_batch_size = 1
gradient_accumulation_steps = 2
optim = 'adamw_hf'
learning_rate = 1e-5
max_grad_norm = 0.3
warmup_ratio = 0.03
lr_scheduler_type = "linear"
num_train_epochs = 3
max_seq_length = 12000

In [0]:
trainer = SFTTrainer(
    model = model,  
    train_dataset = dataset["train"],
    eval_dataset = dataset["test"],
    peft_config = peft_config,
    args = SFTConfig(
      learning_rate = 1e-5, #2e-4,  
      num_train_epochs = num_train_epochs,
      output_dir = base_dir,
      max_seq_length = max_seq_length,
      per_device_train_batch_size = per_device_train_batch_size,
      gradient_accumulation_steps = gradient_accumulation_steps,
      use_liger = True,
      save_strategy = "steps",
      eval_strategy = "steps",
      lr_scheduler_type = lr_scheduler_type,
      optim = optim
      )   
)

trainer.train()

Map:   0%|          | 0/4000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

/local_disk0/.ephemeral_nfs/envs/pythonEnv-151f4588-5515-4975-9984-52aca1cc4862/lib/python3.11/site-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
[rank0]:[W reducer.cpp:1389] Warning: find_unused_parameters=True was specified in DDP constructor, but did not find any unused parameters in the forward pass. This flag results in an extra traversal of the autograd graph every iteration,  which can adversely affect performance. If your model indeed never has any unused parameters in the forward pass, consider turning this flag off. Note that this warning may be a false positive if your model has flow control causing later iterations to have unused parameters. (function operator())


Step,Training Loss,Validation Loss
500,0.435900,0.018688
1000,0.023500,0.016223
1500,0.022300,0.016259
2000,0.020000,0.016056
2500,0.017500,0.015897
3000,0.016400,0.015877
3500,0.016100,0.015907
4000,0.016000,0.016018
4500,0.015900,0.015893
5000,0.015900,0.015864


/local_disk0/.ephemeral_nfs/envs/pythonEnv-151f4588-5515-4975-9984-52aca1cc4862/lib/python3.11/site-packages/peft/utils/save_and_load.py:230: UserWarning: Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.
  warnings.warn("Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.")
/local_disk0/.ephemeral_nfs/envs/pythonEnv-151f4588-5515-4975-9984-52aca1cc4862/lib/python3.11/site-packages/peft/utils/save_and_load.py:230: UserWarning: Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.
  warnings.warn("Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.")
/local_disk0/.ephemeral_nfs/envs/pythonEnv-151f4588-5515-4975-9984-52aca1cc4862/lib/python3.11/site-packages/peft/utils/save_and_load.py:230: UserWarning: Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.
  warnings.warn("Setting `save_embedding_layers` to 

TrainOutput(global_step=6000, training_loss=0.05262324754397074, metrics={'train_runtime': 3190.7527, 'train_samples_per_second': 3.761, 'train_steps_per_second': 1.88, 'total_flos': 9659839131156480.0, 'train_loss': 0.05262324754397074, 'epoch': 3.0})

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [0]:
dbutils.fs.ls("FileStore/churns/llama-3.2-1b/cache")

[FileInfo(path='dbfs:/FileStore/churns/llama-3.2-1b/cache/checkpoint-1000/', name='checkpoint-1000/', size=0, modificationTime=1734729070000),
 FileInfo(path='dbfs:/FileStore/churns/llama-3.2-1b/cache/checkpoint-1350/', name='checkpoint-1350/', size=0, modificationTime=1734729183000),
 FileInfo(path='dbfs:/FileStore/churns/llama-3.2-1b/cache/checkpoint-1500/', name='checkpoint-1500/', size=0, modificationTime=1734734091000),
 FileInfo(path='dbfs:/FileStore/churns/llama-3.2-1b/cache/checkpoint-2000/', name='checkpoint-2000/', size=0, modificationTime=1734734252000),
 FileInfo(path='dbfs:/FileStore/churns/llama-3.2-1b/cache/checkpoint-2500/', name='checkpoint-2500/', size=0, modificationTime=1734734412000),
 FileInfo(path='dbfs:/FileStore/churns/llama-3.2-1b/cache/checkpoint-3000/', name='checkpoint-3000/', size=0, modificationTime=1734734571000),
 FileInfo(path='dbfs:/FileStore/churns/llama-3.2-1b/cache/checkpoint-3500/', name='checkpoint-3500/', size=0, modificationTime=1734734730000),

In [0]:
#pause to decide the best model version

In [0]:
peft_model_checkpoint = 'cache/checkpoint-3000'

In [0]:
# load the checkpoint and merge

tokenizer = AutoTokenizer.from_pretrained(base_model)

base_model_reload= AutoModelForCausalLM.from_pretrained(
    base_model,
    low_cpu_mem_usage=True,
    return_dict=True,
    torch_dtype=torch.float16,
    device_map="auto",
)

#peft_model = PeftModel.from_pretrained(model, peft_model_checkpoint)


if hasattr(tokenizer, "chat_template") and tokenizer.chat_template is not None:
    tokenizer.chat_template = None  # Reset the chat template

base_model_reload, tokenizer = setup_chat_format(base_model_reload, tokenizer)


model = PeftModel.from_pretrained(base_model_reload, peft_model_checkpoint)

model = model.merge_and_unload()

/local_disk0/.ephemeral_nfs/envs/pythonEnv-151f4588-5515-4975-9984-52aca1cc4862/lib/python3.11/site-packages/peft/tuners/tuners_utils.py:543: UserWarning: Model with `tie_word_embeddings=True` and the tied_target_modules=['lm_head'] are part of the adapter. This can lead to complications, for example when merging the adapter or converting your model to formats other than safetensors. See for example https://github.com/huggingface/peft/issues/2018.
  warnings.warn(
/local_disk0/.ephemeral_nfs/envs/pythonEnv-151f4588-5515-4975-9984-52aca1cc4862/lib/python3.11/site-packages/peft/tuners/tuners_utils.py:396: UserWarning: Model with `tie_word_embeddings=True` and the tied_target_modules=['lm_head'] are part of the adapter. This can lead to complications. You can opt to merge the adapter after cloning the weights (to untie the embeddings). You can untie the embeddings by loading the model with `tie_word_embeddings=False`. For example:
```python
from transformers import AutoModelForCausalLM

#

In [0]:
pred = []
for index, row in df_inference.iterrows():
  inputs = tokenizer(row["testing_script"], return_tensors='pt', padding=True, truncation=True).to("cuda")
  outputs = model.generate(**inputs, max_new_tokens=1, num_return_sequences=1)
  text = tokenizer.decode(outputs[0], skip_special_tokens=True)
  answer = text.split("assistant")[1]
  print(f"answer: {answer}")
  pred.append(answer)
  #print("\n")

answer: No
answer: No
answer: No
answer: No
answer: No
answer: No
answer: Yes
answer: Yes
answer: No
answer: No
answer: Yes
answer: Yes
answer: Yes
answer: No
answer: No
answer: No
answer: No
answer: No
answer: No
answer: No
answer: No
answer: Yes
answer: No
answer: Yes
answer: Yes
answer: Yes
answer: No
answer: Yes
answer: No
answer: No
answer: No
answer: Yes
answer: Yes
answer: Yes
answer: No
answer: No
answer: No
answer: No
answer: No
answer: No
answer: Yes
answer: No
answer: No
answer: Yes
answer: No
answer: No
answer: No
answer: No
answer: No
answer: Yes
answer: No
answer: No
answer: No
answer: No
answer: No
answer: Yes
answer: Yes
answer: No
answer: Yes
answer: No
answer: No
answer: No
answer: Yes
answer: Yes
answer: No
answer: No
answer: Yes
answer: No
answer: No
answer: No
answer: No
answer: Yes
answer: Yes
answer: No
answer: Yes
answer: Yes
answer: Yes
answer: Yes
answer: No
answer: No
answer: No
answer: No
answer: Yes
answer: No
answer: Yes
answer: No
answer: No
answer: No
an

In [0]:
gold = df_inference["label"].tolist()

# Sample prediction and gold_answer 
prediction = [1 if x.lower() == "yes" else 0 for x in pred]
gold_answer = [1 if x.lower() == "yes" else 0 for x in gold]

# Compute Precision, Recall, F1 Score 
precision = precision_score(gold_answer, prediction) 
recall = recall_score(gold_answer, prediction) 
f1 = f1_score(gold_answer, prediction) 

# Compute ROC Curve and AUC 
fpr, tpr, _ = roc_curve(gold_answer, prediction) 
roc_auc = auc(fpr, tpr) 

print(f"Precision: {precision:.2f}") 
print(f"Recall: {recall:.2f}") 
print(f"F1 Score: {f1:.2f}") 
print(f"AUC: {roc_auc:.2f}")

Precision: 0.56
Recall: 0.41
F1 Score: 0.48
AUC: 0.54


In [ ]:
# Start an MLflow run 
with mlflow.start_run(): 
  
  # Log metrics 
  mlflow.log_metric("precision", precision) 
  mlflow.log_metric("recall", recall) 
  mlflow.log_metric("f1_score", f1) 
  mlflow.log_metric("roc_auc", roc_auc) 
  
  # Log ROC Curve 
  plt.figure() 
  plt.plot(fpr, tpr, color='darkorange', lw=2, label=f'ROC curve (area = {roc_auc:.2f})') 
  plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--') 
  plt.xlim([0.0, 1.0]) 
  plt.ylim([0.0, 1.05]) 
  plt.xlabel('False Positive Rate') 
  plt.ylabel('True Positive Rate') 
  plt.title('Receiver Operating Characteristic') 
  plt.legend(loc="lower right") 
  plt.savefig("/dbfs/FileStore/roc_curve.png") 
  plt.show() 
  
  # Log ROC curve image 
  mlflow.log_artifact("/dbfs/FileStore/roc_curve.png")

In [0]:
new_model = "/dbfs/FileStore/churns/llama-3.2-1b/model"

model.save_pretrained(new_model)
tokenizer.save_pretrained(new_model)

('/dbfs/FileStore/churns/llama-3.2-1b/model/tokenizer_config.json',
 '/dbfs/FileStore/churns/llama-3.2-1b/model/special_tokens_map.json',
 '/dbfs/FileStore/churns/llama-3.2-1b/model/tokenizer.json')